# Install Libraries

In [ ]:
!pip install pillow

# Download Dataset

In [ ]:
DATASET_DIR = "scalene_triangle_0_355_step5"
!rm -rf "$DATASET_DIR"
!curl -L https://github.com/joshbarrass/Y4ProjectNotebooks/raw/master/scalene_triangle_0_355_step5.zip > "scalene_triangle_0_355_step5.zip"
!unzip -d "$DATASET_DIR" "scalene_triangle_0_355_step5.zip" > /dev/null

# Define Helper Functions

In [ ]:
def normalise(x):
    return x / np.linalg.norm(x)

def from_SO3_to_6D(SO3):
    """Convert an SO(3) rotation matrix to a 6D internal representation."""
    return SO3[:,:2]

def from_6D_to_SO3(D6):
    """Convert a 6D internal representation to an SO(3) rotation matrix."""
    b1 = normalise(D6[:,0])
    b2 = normalise(D6[:,1] - np.dot(b1, D6[:,1])*b1)
    b3 = np.cross(b1, b2)
    return np.array([b1, b2, b3]).T

# Load and Pre-Process Training Data

In [ ]:
import os
import numpy as np
from scipy.spatial.transform import Rotation as R
from PIL import Image, ImageFilter
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

D6_SHAPE = (3, 2)

GAUSS_RADIUS = 3
image_filter = ImageFilter.GaussianBlur(radius=GAUSS_RADIUS)

image_files = sorted(os.listdir(DATASET_DIR))
total_images = len(image_files)
print(f"Dataset contains {total_images} images")
if total_images == 0:
  raise ValueError("dataset has no images")

image_shape = Image.open(os.path.join(DATASET_DIR, image_files[0])).size
print(f"Image shape is {image_shape}")
x_data = np.zeros((total_images,) + image_shape)
y_data = np.zeros((total_images,) + D6_SHAPE)
print(f"x will have shape {x_data.shape}")
print(f"y will have shape {y_data.shape}")
for i, imf in tqdm(enumerate(image_files), total=total_images):
  im = Image.open(os.path.join(DATASET_DIR, imf)).convert("L")
  im = im.filter(image_filter)
  x_data[i,:,:] = np.array(im)
  angle = float(os.path.splitext(imf)[0])
  y_data[i,:,:] = from_SO3_to_6D(R.from_euler("zyx", (angle, 0, 0), True).as_matrix())

plt.imshow(x_data[0,:,:])
print(y_data[0,:,:])